# FPGA acceleration of Merge Sort Algorithm



Author: Fabrizio Maria Aymone

Contact: fabrizio.aymone@gmail.com

*Apache License Version 2.0, January 2004 *;* http://www.apache.org/licenses/*

In [1]:
import time
import numpy as np
np.set_printoptions(threshold=np.inf)

# this is the sequence needed to be sorted
unsorted_sequence = np.random.randint(255, size=1024)
# this is the sequence correctedly sorted
sorted_sequence   = np.sort(unsorted_sequence)

## SW performance

The recursive version of the algorithm does not support sequences of 1024 elements, as it goes into overflow. Therefore, the iterative version is considered.

In [2]:
def merge_sort(arr):
    current_size = 1
    len_arr = len(arr)

    while current_size < len_arr:
        left_start = 0
        while left_start < len_arr:
            mid = min(left_start + current_size - 1, len_arr - 1)
            right_end = min(left_start + 2 * current_size - 1, len_arr - 1)

            merge_subarrays(arr, left_start, mid, right_end)
            left_start += 2 * current_size

        current_size *= 2
    return arr

def merge_subarrays(arr, start, mid, end):
    len_left = mid - start + 1
    len_right = end - mid
    left = [0] * len_left
    right = [0] * len_right

    for i in range(len_left):
        left[i] = arr[start + i]
    for i in range(len_right):
        right[i] = arr[mid + 1 + i]

    i, j, k = 0, 0, start
    while i < len_left and j < len_right:
        if left[i] <= right[j]:
            arr[k] = left[i]
            i += 1
        else:
            arr[k] = right[j]
            j += 1
        k += 1

    while i < len_left:
        arr[k] = left[i]
        i += 1
        k += 1

    while j < len_right:
        arr[k] = right[j]
        j += 1
        k += 1



In [46]:
start_time = time.time()
sw_sorted = merge_sort(unsorted_sequence.astype("uint8"))
stop_time = time.time()
sw_exec_time = stop_time - start_time
assert np.array_equal(sorted_sequence, sw_sorted), "SW sorted array has not been sorted correctly"
print('Software Merge Sort execution time', sw_exec_time)

Software Merge Sort execution time 0.09255146980285645


## FPGA performance

3 different architectures (i.e. tree, row and hybrid) have been designed and benchmarked.

## tree architecture

In [24]:
from pynq import allocate
from pynq import Overlay

overlay = Overlay('/home/xilinx/pynq/overlays/merge_sorter/5.bit')

In [25]:
dma = overlay.axi_dma_0

In [26]:
input_buffer = allocate(shape=(1024,), dtype=np.uint32)
output_buffer = allocate(shape=(1024,), dtype=np.uint32)

In [27]:
for i in range(1024):
    input_buffer[i] = unsorted_sequence[i]

In [28]:
start_time = time.time()
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
hw_sorted=output_buffer
assert np.array_equal(sorted_sequence, hw_sorted), "HW sorted array has not been sorted correctly"

In [29]:
print('Software Merge Sort execution time', sw_exec_time)
print('Hardware Merge Sort execution time: ',hw_exec_time)
print('Hardware acceleration factor: ',sw_exec_time / hw_exec_time)

Software Merge Sort execution time 0.1004939079284668
Hardware Merge Sort execution time:  0.006251335144042969
Hardware acceleration factor:  16.075591151792526


## row architecture

In [31]:
from pynq import allocate
from pynq import Overlay

overlay = Overlay('/home/xilinx/pynq/overlays/merge_sorter/6.bit')

In [32]:
dma = overlay.axi_dma_0

In [33]:
input_buffer = allocate(shape=(1024,), dtype=np.uint32)
output_buffer = allocate(shape=(1024,), dtype=np.uint32)

In [34]:
for i in range(1024):
    input_buffer[i] = unsorted_sequence[i]

In [35]:
start_time = time.time()
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
hw_sorted=output_buffer
assert np.array_equal(sorted_sequence, hw_sorted), "HW sorted array has not been sorted correctly"

In [36]:
print('Software Merge Sort execution time', sw_exec_time)
print('Hardware Merge Sort execution time: ',hw_exec_time)
print('Hardware acceleration factor: ',sw_exec_time / hw_exec_time)

Software Merge Sort execution time 0.1004939079284668
Hardware Merge Sort execution time:  0.006253480911254883
Hardware acceleration factor:  16.07007510770521


## hybrid architecture

In [18]:
from pynq import allocate
from pynq import Overlay

overlay = Overlay('/home/xilinx/pynq/overlays/merge_sorter/merge.bit')

In [19]:
dma = overlay.axi_dma_0

In [20]:
input_buffer = allocate(shape=(1024,), dtype=np.uint32)
output_buffer = allocate(shape=(1024,), dtype=np.uint32)

In [21]:
for i in range(1024):
    input_buffer[i] = unsorted_sequence[i]

In [22]:
start_time = time.time()
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
hw_sorted=output_buffer
assert np.array_equal(sorted_sequence, hw_sorted), "HW sorted array has not been sorted correctly"

In [23]:
print('Software Merge Sort execution time', sw_exec_time)
print('Hardware Merge Sort execution time: ',hw_exec_time)
print('Hardware acceleration factor: ',sw_exec_time / hw_exec_time)

Software Merge Sort execution time 0.1004939079284668
Hardware Merge Sort execution time:  0.003984212875366211
Hardware acceleration factor:  25.223026748848064
